In [1]:
import pandas as pd
import numpy as np

# 1.Data Merge

* concat : 데이터를 위아래로 붙이고, 원할 경우 좌우로 붙이는 것도 가능. 단, 키 값을 정해서 데이터 결합 불가
* merge : column 를 키값으로 설정하여 붙인다
* join : index 를 키값으로 설정하여 붙인다

In [2]:
df_a = pd.DataFrame({'key': ['a','b','c','d','e'], 'num_a': [1,2,3,4,5]})
df_b = pd.DataFrame({'key': ['a','b','c','f','g'], 'num_b': [11,22,33,44,55]})
df_c = pd.DataFrame({'key': ['f','g','h','i','j'], 'num_a': [6,7,8,9,0]})

In [3]:
df_c

,key,num_a
0,f,6
1,g,7
2,h,8
3,i,9
4,j,0


### 1) concat

In [4]:
pd.concat([df_a, df_b])

,key,num_a,num_b
0,a,1.0,NaN
1,b,2.0,NaN
2,c,3.0,NaN
3,d,4.0,NaN
4,e,5.0,NaN
0,a,NaN,11.0
1,b,NaN,22.0
2,c,NaN,33.0
3,f,NaN,44.0
4,g,NaN,55.0


In [5]:
pd.concat([df_a, df_c])

,key,num_a
0,a,1
1,b,2
2,c,3
3,d,4
4,e,5
0,f,6
1,g,7
2,h,8
3,i,9
4,j,0


* pd.concat : 열 이름이 같을 경우 하나의 열로 합쳐지고, 아니면 열이 새로 생긴다

In [6]:
#concat : 기본적으로 위아래로 합쳐진다. axis=1 사용하여 옆으로 붙일 수 있다

pd.concat([df_a, df_b], axis=1)

,key,num_a,key,num_b
0,a,1,a,11
1,b,2,b,22
2,c,3,c,33
3,d,4,f,44
4,e,5,g,55


### 2) merge
* 공통 열이 있으면 suffix 자동 생성
* SQL 스타일의 join 수행 가능 (inner join, left join, right join, outer join)

In [7]:
#df.merge : inner join을 기본으로 한다. 공통으로 있는 값에 대해서만 합쳐지고, 없는 값은 사라진다

df_a.merge(df_b)

,key,num_a,num_b
0,a,1,11
1,b,2,22
2,c,3,33


In [8]:
#how = left, right, outer 으로 지정하여 가능하다, on : 합칠때 기준이 되는 컬럼을 지정해준다 (on을 지정하지 않으면 기본적으로 모든 열 (key, id 등) 이 일치하는 애들을 나타낸다)
#left_on, right_on 으로 각기 다른 열을 지정해줄 수 있다

df_a.merge(df_b, on='key', how='left')

,key,num_a,num_b
0,a,1,11.0
1,b,2,22.0
2,c,3,33.0
3,d,4,NaN
4,e,5,NaN


### 3) Join
* 공통 열이 있으면 suffix 수동 생성 필요

In [9]:
# join은 기준이 되는 열을 index에 두기때문에 그냥 두 개가 붙어진다
df_a.join(df_b, lsuffix='_a', rsuffix='_b')

,key_a,num_a,key_b,num_b
0,a,1,a,11
1,b,2,b,22
2,c,3,c,33
3,d,4,f,44
4,e,5,g,55


In [10]:
df_a = df_a.set_index('key')
df_b = df_b.set_index('key')

In [11]:
#index를 바꿔줘서 join하면 같이 합쳐진다

df_a.join(df_b, how = 'outer')

,num_a,num_b
key,,
a,1.0,11.0
b,2.0,22.0
c,3.0,33.0
d,4.0,NaN
e,5.0,NaN
f,NaN,44.0
g,NaN,55.0


## Practice

In [12]:
salary_1 = pd.read_csv('~/data/salary_1.csv')
salary_2 = pd.read_csv('~/data/salary_2.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/aiffel/data/salary_1.csv'

In [ ]:
salary_1.head()

In [ ]:
salary_2.head()

In [ ]:
#열 이름이 완벽히 같기 때문에, concat을 통해 일단 두 데이터를 합친다
salary_df = pd.concat([salary_1, salary_2])
salary_df

In [ ]:
#index가 중복되는 데이터가 있으므로, 순서대로 다시 정렬해준다

salary_df = salary_df.reset_index(drop=True)

In [ ]:
salary_df

* 연봉 예측할 때 국가별 물가지수 참고하는 것이 도움이 된다

In [ ]:
cpi = pd.read_csv('~/data/cpi.csv')

In [ ]:
cpi.head()

* salary 데이터랑 cpi 데이터의 country 값들이 표기가 조금씩 다르다

In [ ]:
#일단 국가별 명칭의 고유값을 확인해준다

salary_df['Country'].unique()

In [ ]:
cpi['Country'].unique()

In [ ]:
#replace를 통해 국가별 명칭을 일치시켜준다

cpi['Country'] = cpi['Country'].replace({'United States':'USA', 'United Kingdom':'UK'})

In [ ]:
salary_df = salary_df.merge(cpi, on='Country', how='left')

In [ ]:
salary_df.head()

In [ ]:
#불필요한 열들을 삭제해준다

salary_df.drop(['Reference','Previous','Units','Frequency'], axis=1, inplace=True)

In [ ]:
salary_df.head()

In [ ]:
salary_df = salary_df.rename({'Last':'CPI'}, axis=1)

In [ ]:
salary_df.head()

# 2.Missing Value, Outlier / Aggregation and Groupby / Pivot

① Overview : 데이터의 기본적인 특성을 파악하고, 데이터셋의 구조와 내용을 이해하는 데 도움을 줍니다.

head( ): 데이터프레임의 처음 몇 행(기본적으로 5행)을 표시합니다. 데이터의 포맷, 변수의 형태 등을 빠르게 확인할 수 있습니다.  
tail( ): 데이터프레임의 마지막 몇 행을 표시합니다. 데이터의 전체적인 분포를 확인하는 데 도움이 됩니다.  
describe( ): 수치형 열에 대한 기술통계를 제공합니다. 평균, 표준편차, 최소값, 최대값 등을 확인할 수 있습니다.  
info( ): 데이터프레임의 기본 정보를 제공합니다. 각 열의 데이터 타입, 누락된 값의 수, 메모리 사용량 등을 확인할 수 있습니다.  
dtypes: 각 열의 데이터 타입을 보여줍니다. 데이터 형식을 빠르게 이해하는 데 유용합니다.  
  
  
② Outlier : 이상치(Outlier)를 탐지하고 처리하는 데 사용되는 여러 방법과 함수들이 있습니다.  

통계적 방법: Z-점수와 IQR을 사용하여 이상치를 식별할 수 있습니다.  
시각적 방법: 박스 플롯으로 이상치를 시각화하여 이상치를 식별할 수 있습니다.  
데이터 필터링: 조건에 따라 이상치 필터링 및 제거등의 방법으로 이상치를 식별할 수 있습니다.   
  
③ Aggregation : 여러 데이터 포인트를 요약하고, 그룹화하여 새로운 통계 또는 정보를 추출하는 과정입니다.  
   
groupby( ): 데이터를 특정 기준으로 그룹화하여 집계 연산에 적용할 수 있습니다.  
집계 함수: sum( ), mean( ), median( ), min( ), max( ), count( ) 등으로 각 그룹의 요약 통계를 제공 합니다.  
std( ) 함수 : 표준편차는 데이터 세트 내의 값들이 평균으로부터 얼마나 멀리 퍼져 있는지를 측정하는 값으로, 데이터의 분산 정도를 나타냅니다.  
agg( ) 함수: 다양한 집계 함수를 한 번에 적용하여 다양한 요약 통계를 얻을 수 있습니다.  
  
  
④ Pivot Table  
  
데이터프레임을 재구성하여, 특정 열을 새로운 행과 열의 인덱스로 사용하고, 다른 열의 데이터로 새로운 표를 만드는 데 사용됩니다.


In [ ]:
#data type확인한다
salary_df.info()

* cpi는 다 숫자값이어야 하는데 object로 생길 수도 있다. 문자값이 섞여있거나, 그냥 그렇게 뜰 수 있음. 데이터타입 바꿔주기

In [ ]:
salary_df['CPI'] = pd.to_numeric(salary_df['CPI'])

In [ ]:
salary_df.describe()

* 경력값 : min, max 이상함
* salary : min이 이상함


## 1) Missing Value

In [ ]:
salary_df.isna().sum()

In [ ]:
#Age null값 데이터 확인하기
salary_df[salary_df['Age'].isna()]

In [ ]:
#Null값 삭제해주기
salary_df = salary_df.dropna()

In [ ]:
salary_df[salary_df['Years of Experience'] == -1]

In [ ]:
#경력 연수 이상한 사람 삭제해주기
salary_df = salary_df[salary_df['Years of Experience'] != -1]

In [ ]:
salary_df['Years of Experience'].sort_values()

In [ ]:
salary_df[salary_df['Years of Experience'] == 82]

* 나이가 25살인데 경력이 82년이니까 이상하다

In [ ]:
#경력이 나이보다 많은 경우는 이상하니까 걸러본다 (~로 거를 수 있다)
salary_df = salary_df[~(salary_df['Age'] -18  < salary_df['Years of Experience'])]

In [ ]:
salary_df.describe()

In [ ]:
salary_df[salary_df['Years of Experience'] ==0]

* 경력이 0년인거는 정상인 것 같으니 넘어간다

## 2) Group By

In [ ]:
salary_df[salary_df['Gender'] == 'Male']['Salary'].mean()

In [ ]:
#숫자로 연산이 되는 데이터만 표출된다
salary_df.groupby('Gender').mean()

In [ ]:
#보고싶은 열만 지정할 수도 있다
salary_df.groupby('Gender')['Salary'].mean()

In [ ]:
#두 개의 열로 묶어줄 수도 있다
salary_df.groupby(['Gender','Country'])['Salary'].mean()

In [ ]:
#index로 묶어주면 이렇게 데이터 테이블로 나온다. 매우 유용한 문법
salary_df.groupby(['Gender','Country'])['Salary'].mean().reset_index()

In [ ]:
#agg를 통해 다양한 집계함수를 한 번에 적용할 수 있다
salary_df.groupby('Gender')['Salary'].agg(['sum','mean'])

## 3) Pivot Table

In [ ]:
#피봇테이블 만들기 (데이터, 행, 열, 값, 연산)
pd.pivot_table(salary_df, index='Gender', columns='Country', values='Salary', aggfunc= np.mean)

In [ ]:
pd.pivot_table(salary_df, index=['Gender','Race'], columns='Country', values='Salary', aggfunc= sum)

### (참고) Pivot 만들기

In [ ]:
sales_df = pd.DataFrame({'company': ['a','a','a','a','b','b','b','b'],
             'quarter': ['q1','q2','q3','q4','q1','q2','q3','q4'],
             'sales': [111,222,333,444,555,666,777,888]})

In [ ]:
sales_temp = pd.pivot(sales_df, index= 'company', columns= 'quarter', values='sales')

In [ ]:
#quarter 라는 column name을 삭제해ㅅ준다
sales_temp.columns = sales_temp.columns.rename('')

In [ ]:
new_sales_df = sales_temp.reset_index()

In [ ]:
new_sales_df

## 4) Melt

In [ ]:
#pivot table형태를 이렇게 바꾸고 싶을 때 
pd.melt(new_sales_df, id_vars='company', value_vars=['q1','q2','q3','q4'], var_name='quarter', value_name='sales').sort_values('company')

# 3. Log, One-Hot Encoding
① Log Transformation : 데이터의 변환을 위해 로그 함수를 적용하는 과정. 데이터의 스케일을 조정하거나 정규성을 높이는 데 사용됩니다.  
② get_dummies : 주어진 범주형 열의 각 고유 범주를 대표하는 새로운 이진(0 또는 1) 열을 생성합니다

## 1) Log

In [ ]:
#로그 : 10의 몇 승인지 나온다. 기본적으로는 자연로그를 취한다
np.log10(100)

In [ ]:
#가격이 기하급수적으로 오르는 데이터를 가정한다
price_df = pd.DataFrame({'level':[1,2,3,4,5,6,7], 'price':[1,10,100,1000,10000,100000,1000000]})

In [ ]:
price_df

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(x='level',y='price', data=price_df)

In [ ]:
sns.scatterplot(x=price_df['level'], y=np.log(price_df['price']))

* Linear Regression 에서는 독립변수, 종속변수가 서로 선형관계에 있을수록 예측에 유리하다. 이럴 때 Log를 유용하게 취해줄 수 있다

In [ ]:
np.log(55)

In [ ]:
np.exp(4.007333185232471)

## 2) One-Hot Encoding

In [ ]:
salary_df.head()

* male, female을 원핫인코딩 하면, 0과 1로 나뉜다. 열이 새로 생긴다

In [ ]:
#컬럼을 생략하지 않고 다 보여주게 만드는 명령어. 보통 100개 넘어가는 컬럼은 선언하지 않는다
pd.set_option('display.max_columns', 50)

In [ ]:
#drop 해주는 이유는, 하나만 있어도 연산에 문제가 없기 때문이다
pd.get_dummies(salary_df, columns=['Gender','Country','Race'], drop_first=True)

In [ ]:
#각 값이 몇개 있는지
salary_df[['Gender','Country','Race','Job Title']].nunique()

In [ ]:
salary_df['Job Title'].value_counts().tail(20)

In [ ]:
salary_df['Job Title'].unique()

In [ ]:
job = pd.read_csv('~/data/job.csv')

In [ ]:
salary_df = salary_df.merge(job, on='Job Title', how='left')

In [ ]:
salary_df.drop('Job Title', axis=1, inplace=True)

In [ ]:
salary_df

In [ ]:
salary_df['Jobs'].value_counts()

In [ ]:
salary_df = pd.get_dummies(salary_df, columns=['Gender','Country','Race','Jobs'], drop_first=True)

In [ ]:
salary_df

# 4. Scaling


① Scaling : 데이터의 범위를 조정하는 과정. MinMaxScaler, StandardScaler 등을 사용해 특성의 스케일을 조정 합니다.  

- 거리기반 모델을 쓸때는 반드시 스케일링 써서 각각 변수의 특성에 따라 동일한 스케일을 지닐수 있게 맞춰주는것이 중요 합니다.  
  
  
② 주요 Scaling 방법 :

- Standardization : (x-mean/ std) 결과적으로 데이터는 평균이 0이고 표준편차가 1인 분포를 갖게 됩니다.  
- Robust Scaling : (x-Q2) / (Q3-Q1) 이 방법은 중앙값과 사분위 범위를 사용하여 데이터를 스케일링합니다. 표준화와 유사하지만, 이상치의 영향을 덜 받습니다.  
- MinMaxScaler : (x-min)/ (max-min) 데이터를 0과 1 사이의 범위로 조정합니다. 주로 최소값과 최대값을 사용하여 계산합니다. ex)Deep Learning  
  
  
③ 각 Scaling 방법의 적합한 상황과 특징:  

- Standardization : 아웃라이어가 없는 경우, 변수의 분포가 정규분포를 따를 때 적합합니다.  
리니어 리그레이션(연속적인 값 예측), 로지스틱 회귀(분류 문제에 적합한 확률 결과 제공), 서포트 벡터 머신(이진 또는 다중 클래스 분류, 회귀)에 사용됩니다.  
주의사항 : 아웃라이어에 민감하므로, 아웃라이어가 존재하지 않을 때 더 효과적입니다.  
- Robust Scaling : 아웃라이어가 많은 데이터에 적합 합니다.  
특징 : 정규분포를 따르지 않는 데이터에 적용하기 좋은 방법입니다.  
- MinMaxScaler : 데이터의 범위를 0에서 1로 제한하고자 할 때 적합 합니다.주로 딥러닝 모델에서 사용됩니다.  
특징 : 스케일링 범위를 제한하고자 할 때 유용합니다.  
  
각 변수의 크기(Scale) 중요할 때 사용한다 : K-Means, KNN(거리 기반 모델링) 등

## 1) Standard Scaling

In [ ]:
salary_df['Age'].mean()

In [ ]:
salary_df['Age'].std()

In [ ]:
(salary_df['Age']- salary_df['Age'].mean()) / salary_df['Age'].std()

## 2) Robust Scaling 

In [ ]:
salary_df['Age'].quantile(0.75)

In [ ]:
salary_df['Age'] - salary_df['Age'].quantile(0.5) / (salary_df['Age'].quantile(0.75) - salary_df['Age'].quantile(0.25))

## 3) MinMax Scaling

In [ ]:
(salary_df['Age']- salary_df['Age'].min())/ (salary_df['Age'].max() - salary_df['Age'].min())

* 근데 이 모든 scaling을 쉽게 해주는 라이브러리가 있음! 

In [ ]:
#sklearn 다 부르면 용량이 많으니까, 용량때문에 필요한 라이브러리만 취사선택해서 가져오는 게 더 효율적임
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [ ]:
ss = StandardScaler()
rs = RobustScaler()
mm = MinMaxScaler()

### 1) Standard

In [ ]:
#1) 필요한 데이터를 학습시켜준다. (ss의 경우, 평균, 표준편차를 학습한다)
ss.fit(salary_df)

In [ ]:
#2) 학습된 정보로 연산을 해준다
ss.transform(salary_df)

* 위처럼 numpy array 형태로 나오고, 열 이름도 없으므로, 아래와 같이 dataframe형식으로 지정해줄 수 있다 

In [ ]:
ss_df = pd.DataFrame(ss.transform(salary_df), columns= salary_df.columns)

In [ ]:
ss_df.head()

### 2) Robust

In [ ]:
rs.fit(salary_df)

In [ ]:
rs_df = pd.DataFrame(rs.transform(salary_df),columns= salary_df.columns)

In [ ]:
rs_df.head()

### 3) MM

In [ ]:
mm.fit(salary_df)

In [ ]:
mm_df = pd.DataFrame(mm.transform(salary_df), columns= salary_df.columns)

In [ ]:
mm_df.head()

In [ ]:
ss_df.describe()

In [ ]:
rs_df.describe()

In [ ]:
mm_df.describe()

### 참고) fit, transform 한번에 하는 방법

In [ ]:
ss.fit_transform(salary_df)

# 5. PCA analysis
* 정보의 손실을 최소화하는 방향으로, 데이터의 차원을 축소하는 기법
* 목적 : 변수의 갯수를 줄이기 위해, 차원을 축소하기 위해
* 두 변수의 상관관계가 높으면 하나 없애주는 게 낫다
* 장점 : 시각화할 때 유용하다
* 단점 : 주성분 변수가 어떤 의미, 특성을 가지고 있는지 아무 정보를 가지고 있지 않음

① PCA (Principal Component Analysis) : 차원 축소를 위한 기술. 데이터의 주요 특성을 유지하면서 차원을 줄여 계산 효율성을 높입니다.  
② explained_variance_ratio_ : 주로 주성분 분석(Principal Component Analysis, PCA)과 같은 차원 축소 기법에서 사용되는 속성입니다. 이 속성은 각 주성분이 원본 데이터의 분산(variance)을 얼마나 설명하는지에 대한 비율을 나타냅니다.  


## 1) PCA 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

In [ ]:
#1)학습시키기
pca.fit(salary_df)

In [ ]:
#2)학습된 정보로 연산하기
pd.DataFrame(pca.transform(salary_df))

In [ ]:
#2개의 주성분으로만 학습시킨다
pca = PCA(2)

In [ ]:
pd.DataFrame(pca.fit_transform(salary_df), columns=['PC1', 'PC2'])

In [ ]:
#거의 대부분의 정보를 두 개의 주성분이 가지고 있음을 확인
(pca.explained_variance_ratio_).sum()

## 2) 상관관계 확인하기

In [13]:
salary_df.corr()

NameError: name 'salary_df' is not defined

* 나이, 경력이 상관관계가 높으니까 걸러주는 식으로 가능

# 6. Quiz

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
# salary_1 이름으로 salary_1.csv 불러오기 (데이터 위치: 'data/salary_1.csv')
salary_1 = pd.read_csv('~/data/salary_1.csv')

In [ ]:
salary_1

In [ ]:
salary_2 = pd.read_csv('~/data/salary_2.csv')

In [ ]:
salary_2

In [ ]:
# salary_1 과 salary_2를 위/아래로 붙이고, salary_df 이름으로 저장하기
salary_df = pd.concat([salary_1, salary_2])

In [ ]:
salary_df

In [ ]:
# salary_df의 결측치 비율 확인하기
salary_df.isna().mean()

In [ ]:
# 결측치 행 제거하기
salary_df =  salary_df.dropna()

In [ ]:
# Gender별 Salary의 평균(mean)을 구해 gender_salary 로 저장하기 
gender_salary = salary_df.groupby('Gender')['Salary'].mean()

In [ ]:
# gender_salary의 인덱스(Gender)를 컬럼으로 전환하여 저장하기
gender_salary =  gender_salary.reset_index()

In [ ]:
gender_salary

In [ ]:
# Gender를 기준으로 하여, salary_df에 gender_salary를 붙여서(left join) salary_df로 저장하기
salary_df = salary_df.merge(gender_salary, on='Gender', how='left')

In [ ]:
salary_df

In [ ]:
# 컬럼이름 변경: "Salary_x"를 "Salary"로, "Salary_y"를 "Gender_salary"로 변경하여 저장
salary_df = salary_df.rename({'Salary_x': 'Salary', 'Salary_y': 'Gender_salary'}, axis=1)

In [ ]:
# 다음 기준으로 Pivot Table 만들기 -> 행: Country, 열: Gender, 값: Years of Experience
pd.pivot_table(salary_df, index='Country', columns='Gender', values='Years of Experience')

In [ ]:
# Salary 변수에 로그를 취하여 'Salary_log'로 저장하기
salary_df['Salary_log'] = np.log(salary_df['Salary'])

In [ ]:
# 변수 중 Data Type이 object인 변수들 제거하고 저장하기
salary_df.info()

In [ ]:
salary_df.drop(['Gender','Job Title','Country','Race'], axis=1, inplace=True)

In [ ]:
# RobustScaler 패키지 불러오기
from sklearn.preprocessing import RobustScaler

In [ ]:
rs = RobustScaler()

In [ ]:
rs.fit(salary_df)

In [ ]:
rs_df = rs.transform(salary_df)

In [ ]:
# rs_df를 Pandas DataFrame으로 변경하여 저장 (컬럼 이름도 기존 컬럼이름으로 채워넣기)
rs_df =  pd.DataFrame(rs_df, columns= salary_df.columns)

In [ ]:
salary_df.columns

In [ ]:
# salary_df에서 주성분을 뽑을 경우, 최대로 뽑을 수 있는 주성분 개수는?
salary_df.columns.nunique()

In [ ]:
# PCA 패키지 불러오기
from sklearn.decomposition import PCA

In [ ]:
# PCA를 사용하기 위해 pca 이름으로 저장: 2개의 주성분을 뽑을 수 있도록 설정
pca = PCA(2)

In [ ]:
# pca로 salary_df를 학습 및 변환하여 pca_df로 저장
pca_df =  pca.fit_transform(salary_df)

In [ ]:
# pca_df를 Pandas DataFrame으로 변경하고, 각 컬럼이름을 PC1, PC2로 설정하여 pca_df로 저장
pca_df = pd.DataFrame(pca_df, columns=['PC1','PC2'])

In [ ]:
# 추출된 두개의 주성분으로 기존 데이터 정보의 얼마만큼을 설명할 수 있는지 확인하는 코드 작성
(pca.explained_variance_ratio_).sum()